# Modules

In [5]:
%load_ext autoreload
%autoreload 2

from pdathome.constants import global_constants as gc
from pdathome.classification import train_test_filtering_gait, store_filtering_gait

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Process data

In [6]:
classifier_names = [gc.classifiers.LOGISTIC_REGRESSION]
gsearch = False

In [7]:
# for subject in gc.participant_ids.PD_IDS:
#     train_test_filtering_gait(subject, gsearch=gsearch, classifier_names=classifier_names)

store_filtering_gait(classifier_names=classifier_names)

Storing model output at arm_activity step for classifier logreg ...
